# Finding Coincident NASA Airborne and Orbital Data

**Summary**

Often, novel remote sensing research requires utilizing data from multiple instruments. This Jupyter notebook provides an example of finding coincident data from NASA Earth Science missions. Users will learn how to submit mutiple queries to NASA's CMR using the *earthaccess* Python library to accomplish this goal. These queries can search the NASA Earthdata Archive of over 120 petabytes using keywords, spatial constraints (point, bounding box, or polygon), and temporal constraints. From a basic search for AVIRIS-3, we take the metadata returned by the query and use it to define spatial and temporal constraints for a second and third seach to to find orbital data from the ECOSTRESS and EMIT collections.

<div>
<img src="../img/coincident_airborne_orbital.png" width="750"/>
</div>

**Background**

The **AVIRIS-3** instrument is an airborne imaging spectrometer that measures light in visible and infrared wavelengths. These measurements display unique spectral signatures that correspond to the chemical composition on the Earth's surface and in the atmospheric column above. There are several applications for this data ranging from surface mineral exploration to agriculture. Recently, the instrument was flown over wildfires in Alabama and provided realtime support to firefighters. More specifics about the AVIRIS-3 mission can be found on the [AVIRIS-3 website](https://earth.jpl.nasa.gov/estd-missions/airborne/aviris-3/) and [AVIRIS-3 dataset landing pages](https://daac.ornl.gov/cgi-bin/dataset_lister.pl?p=47).

The **ECOSTRESS** instrument is a multispectral thermal imaging radiometer designed to answer three overarching science questions:

- How is the terrestrial biosphere responding to changes in water availability?
- How do changes in diurnal vegetation water stress  the global carbon cycle?
- Can agricultural vulnerability be reduced through advanced monitoring of agricultural water consumptive use and improved drought   estimation?

The ECOSTRESS mission is answering these questions by accurately measuring the temperature of plants.  Plants regulate their temperature by releasing water through tiny pores on their leaves called stomata. If they have sufficient water they can maintain their temperature, but if there is insufficient water, their temperatures rise and this temperature rise can be measured with ECOSTRESS. The images acquired by ECOSTRESS are the most detailed temperature images of the surface ever acquired from space and can be used to measure the temperature of an individual farmers field.

More details about ECOSTRESS and its associated products can be found on the [ECOSTRESS website](https://ecostress.jpl.nasa.gov/) and [ECOSTRESS product pages](https://lpdaac.usgs.gov/product_search/?query=ECOSTRESS&status=Operational&view=cards&sort=title) hosted by the Land Processes Distributed Active Archive Center (LP DAAC).

The **EMIT** instrument is also an imaging spectrometer, located on the international space station. The EMIT mission focuses specifically on mapping the composition of minerals to better understand the effects of mineral dust throughout the Earth system and human populations now and in the future. In addition, the EMIT instrument can be used in other applications, such as mapping of greenhouse gases, snow properties, and water resources.

More details about EMIT and its associated products can be found on the [EMIT website](https://earth.jpl.nasa.gov/emit/) and [EMIT product pages](https://lpdaac.usgs.gov/product_search/?query=EMIT&status=Operational&view=cards&sort=title) hosted by the LP DAAC.

**Requirements**
 - [NASA Earthdata Account](https://urs.earthdata.nasa.gov/home)   
 - *No Python setup requirements if connected to the workshop cloud instance!*  
 - **Local Only -** Set up Python Environment - See **setup_instructions.md** in the `/setup/` folder to set up a local compatible Python environment 

**Tutorial Outline**

1. Searching for Data
2. Wrangling UMM Metadata
3. Visualizing Results
4. Second Search (ECOSTRESS)
5. Third Search (EMIT)
6. Downloading or Streaming Data

In [ ]:
import numpy as np
import earthaccess
from shapely.geometry import Polygon, polygon
from shapely.ops import unary_union
import folium
import copy
from branca.element import Figure

## 1. Searching for Data

To search for data we'll use the `earthaccess` Python library. `earthaccess` simplifies the ammount of code required to search for data, and handles authentication. You do not need to authenticate to search, but do for downloading or streaming data. We'll go ahead and use the `login` function to add our credentials to this session. This function will retrieve your login info from a `.netrc` file if one exists, or prompt you for username and password and create one if you use the `persist` argument.

In [ ]:
auth = earthaccess.login(persist=True)

For the first step of our initail search for AVIRIS-3 data, we can use `earthaccess` to conduct a query for "AVIRIS-3" to find information about the data collections (often referred to as data products or data sets) associated with that keyword. From this search, we will get the unique `concept-id` associated with the data collection we want to use, AVIRIS-3 L2A Orthocorrected Surface Reflectance, Facility Instrument Collection. A `concept-id` is a unique identifier for a collection, granule, or a service provided by NASA. These types are indicated by a leading C, G, or S respectively. Using the collection `concept-id` for AVIRIS-3, we can find granules (scenes) from that collection.

This query returns a list of dictionaries, from which we will specify to return 3 fields: "Shortname", "EntryTitle", and "Version". These fields provide information that will help us select `concept-id` we want to use in our search.

In [ ]:
# AVIRIS Collection Query
aviris_collection_query = earthaccess.collection_query().keyword('AVIRIS-3')
# Retrieve Relevant Information
aviris_collection_query.fields(['Shortname', 'EntryTitle','Version']).get()

Now that we know the `concept-id` for the AVIRIS-3 L2A Orthocorrected Surface Reflectance collection, we can provide it was an agrument for our data search. 

In [ ]:
# Conduct Search
results_airborne = earthaccess.search_data(concept_id="C3369603199-ORNL_CLOUD", count=1000)
print(f"Granules Found: {len(results_airborne)}")

## 2. Wrangling UMM Metadata

The results from our search for AVIRIS-3 data are returned as a list. By selecting an item in the list and using the `umm` key, we can see a nested dictionary of all the metadata for that a specific granule.

In [ ]:
results_airborne[0]['umm']

We can pull information like temporal and spatial extent from the metadata. All data in CMR either have a temporal extent, which can be either a RangeDateTime or SingleDateTime. For most airborne and orbital data these are a RangeDateTime. Based on this knowledge and the above metadata, we can build a function to show us the `BeginningDateTime` from all of the scenes. We will then use that function to build a unique list of dates for the airborne campaign. With this list of dates we can plan additional searches.

In [ ]:
# Get Unique Dates
def get_beginning_dt(result):
    return result['umm']['TemporalExtent']['RangeDateTime']['BeginningDateTime']

timestamps = [get_beginning_dt(result) for result in results_airborne]
unique_dates = set([timestamp.split("T")[0] for timestamp in timestamps])
print(unique_dates)

After creating this set of dates, we can use it to group our results. This will help us visualize the data, and set up our strategy for finding orbital data within a specified time window of the flights.

In [ ]:
# Group Results By Date
results_by_date = {
    date: [
        result
        for result in results_airborne
        if date in get_beginning_dt(result)
    ]
    for date in unique_dates
}

As mentioned previously, the metadata for each granule also has the spatial extent associated with the acquisition. We can wrangle this into an appropriate format for additional searches, and visualizations.

## 3. Visualizing Results

For visualizing the results, we'll import a `get_vertices` function from the `tutorial_util` module contained in this repository. This function pulls the spatial information out of the results and formats it as a list of polygon vertices so it can be used for `folium` plots or searches with `earthaccess`.

In [ ]:
from modules.search_util import get_vertices

Use folium and our `get_vertices` function to plot the geometry of each acquisition, grouped by date.

In [ ]:
# Visualize our Airborne Campaign Location
fig = Figure(width="750px", height="375px")
# Create Map
m = folium.Map(tiles=None)
fig.add_child(m)

# Add Basemap
folium.TileLayer(
    tiles=(
        "https://server.arcgisonline.com/ArcGIS/rest/"
        "services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
    ),
    name="ESRI Satellite",
    attr="Esri",
    overlay=False,
    control=True
).add_to(m)

# Define Colormap List (hex) - (https://colorbrewer2.org/#type=qualitative&scheme=Set3&n=12)
cmap = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd','#ccebc5','#ffed6f']

# Add Flightline Polygons by Date
for i, date in enumerate(sorted(results_by_date.keys())):
    # Create FeatureGroup for a Date
    fg = folium.FeatureGroup(name=f"{date} AVIRIS-3 Flightlines")
    color = cmap[i % len(cmap)]
    for record in results_by_date[date]:
        # Define Tooltip
        tooltip_dict = {"Granule ID": record['umm']['GranuleUR'], **record['umm']['TemporalExtent']['RangeDateTime']}
        html = "<br>".join(f"<b>{k}</b>: {v}" for k, v in tooltip_dict.items())
        # Add Polygon
        folium.Polygon(
                locations=get_vertices(record, lat_lon=True),
                color=color,
                weight=2,
                fill_opacity=.2,
                tooltip=html
            ).add_to(fg)
    fg.add_to(m)
# Set Bounds and Add Layer Control Widget
m.fit_bounds(bounds=fg.get_bounds())
folium.LayerControl().add_to(m)
fig

We can pick a date here, for example **2024-09-05** and then search for data collections that fall within the same spatial area during a desired timeframe. Let's see what ECOSTRESS data that falls within a week of the field and airborne overpass from **2024-09-05**. 

First, lets simplify our search region. Instead of searching for all the flightline polygons on that date, we can merge intersecting ones to conduct fewer queries. If you have a very large number (100s-1000s), the queries can start to take a long time. Loop over our results on the selected date, retrieving the vertices, and putting them in into a shapely polygon to simplify using a unary union. This process will merge all touching polygons and provide us with a list of merged geometries.

> Note: The spatial extent could be further simplified into a convex hull if that makes more sense for your application, or you have a resulting polygon with too many vertices. There is a limit of ~16,000 vetices.

In [ ]:
selected_date = '2024-09-05'

In [ ]:
polys = []
for result in results_by_date[selected_date]:
    coords = get_vertices(result, lat_lon=False)
    poly = polygon.orient(Polygon(coords), sign=1.0)
    polys.append(poly)
merged = unary_union(polys)
merged

When searching for a polygon using `earthaccess` it expects a list of coordinates that form a ring in counter-clockwise order. To accomplish this we'll make them into a list using list comprehension and the `orient` function from `shapely.polygon`.

In [ ]:
merged_rois = [list(polygon.orient(poly,sign=1.0).exterior.coords) for poly in list(merged.geoms)]

After merging, we've reduced the geometries down to 2, meaning only 2 search queries are required. Now add these merged polygons to our existing map.

In [ ]:
# Add Merged Flightlines to existing figure
for roi in merged_rois:
    folium.Polygon(
        name="Merged AVIRIS-3 Granules",
        locations=[(lat,lon) for lon, lat in roi],
        color='black',
        tooltip=f'Merged AVIRIS-3 Granules Footprint: {selected_date}'
    ).add_to(m)

fig

## 4. Second Search

Now that we have our spatial and temporal constraints for our orbital data search, we just need to retrieve the concept-id for the collection we want.

In [ ]:
# ECOSTRESS Collection Query
eco_collection_query = earthaccess.collection_query().keyword('ECOSTRESS L2T')
eco_collection_query.fields(['Shortname', 'EntryTitle','Version']).get()

Now we can select the concept-id and query for our data. To do this, we'll search for each roi and combine the results. One thing to note here is that since the rois we have provided are small relative to ECOSTRESS scenes, the same scene will likely be a result for each search. To stop us from having duplicates in our results, we'll omit granules that are already present in our results.

In [ ]:
temporal_range = ('2024-08-25T00:00:00.00','2024-09-15T23:59:59.99')

In [ ]:
# Orbital Search Queries
eco_results = []
seen = set()
# Check each ROI
for roi in merged_rois:
    search_params = {
        "concept_id":"C2076090826-LPCLOUD",
        "temporal":temporal_range,
        "polygon": roi
    }
    results_part = earthaccess.search_data(**search_params)
    # Ensure duplicates are not added
    for item in results_part:
        if item not in seen:
            seen.add(item)
            eco_results.append(item)
print(f"Granules Found: {len(eco_results)}")

> This is kind of slow, and can be implemented in parallel or using asynchronous requests. Just make sure you're not overwhelming the service with the quantity of requests.

Now, import the `get_asset_urls` helper function from the `search_util` module to get urls associated with results. It can be used to retrieve links to data or browse imagery (quicklooks). 

In [ ]:
from modules.search_util import get_asset_urls

Now visualize our ECOSTRESS results. We'll use the above function in our `folium` plot to visualize the browse imagery for ECOSTRESS scenes.

> Note: Not all data collections have browse imagery for each granule, and some that do are not designed to be plotted using the extent geometry, like we do below.

In [ ]:
# Visualize our Airborne Campaign Location
eco_fig = Figure(width="750px", height="375px")

# Create Map
eco_map = folium.Map(tiles=None)
eco_fig.add_child(eco_map)

# Add Basemap
folium.TileLayer(
    tiles=(
        "https://server.arcgisonline.com/ArcGIS/rest/"
        "services/World_Imagery/MapServer/tile/{z}/{y}/{x}"
    ),
    name="ESRI Satellite",
    attr="Esri",
    overlay=False,
    control=True
).add_to(eco_map)

# Define Colormap List (hex) - (https://colorbrewer2.org/#type=qualitative&scheme=Set3&n=12)
cmap = ['#8dd3c7','#ffffb3','#bebada','#fb8072','#80b1d3','#fdb462','#b3de69','#fccde5','#d9d9d9','#bc80bd','#ccebc5','#ffed6f']

# Add Flightline Polygons by Date
for i, result in enumerate(eco_results):

    # Create tooltip for each feature
    tooltip_meta = {"Granule ID": result['umm']['GranuleUR'], **result['umm']['TemporalExtent']['RangeDateTime']}

    # Convert tooltip to html
    html = "<br>".join(f"<b>{k}</b>: {v}" for k, v in tooltip_meta.items())

    # Create FeatureGroup
    fg = folium.FeatureGroup(name=tooltip_meta['Granule ID'])

    # Get Coordinates and Color
    coords = get_vertices(result, lat_lon=True)
    color = cmap[i % len(cmap)]

    # Add Polygons
    result_poly = folium.Polygon(
        locations=coords,
        color=color,
        weight=2,
        fill=True,
        fill_opacity=0,
        tooltip=html
        ).add_to(fg)
    fg.add_to(eco_map)

    # Add Browse Image
    bounds = result_poly.get_bounds()
    folium.raster_layers.ImageOverlay(
            image=get_asset_urls(result, extension=".png", first_only=True),
            bounds=bounds,
            opacity=0.75,
            zindex=1,
        ).add_to(fg)
    
# Add Merged ROIs from Search
for roi in merged_rois:
    folium.Polygon(locations=[(lat,lon) for lon, lat in roi],
               color='black').add_to(eco_map)

# Set Bounds and Add Layer Control Widget
eco_map.fit_bounds(bounds=fg.get_bounds())
folium.LayerControl().add_to(eco_map)
eco_fig

## 5. Third Search

Similarly to our ECOSTRESS Search, we can do the same for EMIT L2A Reflectance. Conduct a collection query to retrieve the `concept-id`, then search for data like we previously did for ecostress.

In [ ]:
# ECOSTRESS Collection Query
emit_collection_query = earthaccess.collection_query().keyword('EMIT L2A')
emit_collection_query.fields(['Shortname', 'EntryTitle','Version']).get()

In [ ]:
# Orbital Search Queries
emit_results = []
seen = set()
# Check each ROI
for roi in merged_rois:
    search_params = {
        "concept_id":"C2408750690-LPCLOUD",
        "temporal":temporal_range,
        "polygon": roi
    }
    results_part = earthaccess.search_data(**search_params)
    # Ensure duplicates are not added
    for item in results_part:
        if item not in seen:
            seen.add(item)
            emit_results.append(item)
print(f"Granules Found: {len(emit_results)}")

Build the same type of `folium` figure.

In [ ]:
# Visualize our Airborne Campaign Location
emit_eco_fig = Figure(width="750px", height="375px")
emit_eco_map = copy.deepcopy(eco_map)

# Remove old Layer Control
for key, child in list(emit_eco_map._children.items()):
    if isinstance(child, folium.LayerControl):
        emit_eco_map._children.pop(key)

# Add New Map to Figure
emit_eco_fig.add_child(emit_eco_map)

# Define Colormap List (hex) - (https://colorbrewer2.org/#type=qualitative&scheme=Paired&n=12)
cmap_emit = ['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6','#6a3d9a','#ffff99','#b15928']

# Add Flightline Polygons by Date
for i, result in enumerate(emit_results):

    # Create tooltip for each feature
    tooltip_meta = {"Granule ID": result['umm']['GranuleUR'], **result['umm']['TemporalExtent']['RangeDateTime']}

    # Convert tooltip to html
    html = "<br>".join(f"<b>{k}</b>: {v}" for k, v in tooltip_meta.items())

    # Create FeatureGroup
    fg = folium.FeatureGroup(name=tooltip_meta['Granule ID'])

    # Get Coordinates and Color
    coords = get_vertices(result, lat_lon=True)
    color = cmap_emit[i % len(cmap_emit)]

    # Add Polygons
    result_poly = folium.Polygon(
        locations=coords,
        color=color,
        weight=2,
        fill=True,
        fill_opacity=0,
        tooltip=html
        ).add_to(fg)
    fg.add_to(emit_eco_map)

    # No Browse Image for EMIT since its not orthorectified

# Set Bounds and Add Layer Control Widget
emit_eco_map.fit_bounds(bounds=fg.get_bounds())
folium.LayerControl().add_to(emit_eco_map)
emit_eco_fig

## 5. Selecting Assets and Downloading or Streaming

We can view what types of files are available by looking in the `RelatedUrls` dictionary. 

In [ ]:
results_by_date['2024-09-05'][0]['umm']['RelatedUrls']

The `get_asset_urls` retrieves just the URLs from this dictionary using string matching to help filter what assets we want. For example we can look at examples of all of the links:

In [ ]:
get_asset_urls(results_by_date['2024-09-05'][0])

Or we can grab just the reflectance .nc files.

In [ ]:
get_asset_urls(results_by_date['2024-09-05'][0], extension=".nc", contains="_RFL_ORT")

Note that since our extension and contains arguments are just using string matching, any string appeearing in all of the filenames, like RFL without an _ will return all of the urls. 

We can use this function on our lists of results to build a simple list of urls for the assets we want.

In [ ]:
# Get Results for a specific Date
aviris_links = [
    url for result in results_by_date['2024-09-05']
    for url in get_asset_urls(result, extension=[".nc"])
]
len(aviris_links)

In [ ]:
aviris_links[:5]

In [ ]:
eco_links = [
    url for result in eco_results
    for url in get_asset_urls(result, contains=["_LST.", "_cloud"], extension=".tif")
    ]
len(eco_links)

In [ ]:
eco_links[:5]

In [ ]:
emit_links = [
    url for result in emit_results
    for url in get_asset_urls(result, contains=["_RFL_"],extension=".nc")
]
len(emit_links)

In [ ]:
emit_links

# 6. Downloading or Streaming Data

Depending on the objectives of your workflow and internet speed you can choose to download or stream the data. In this notebook we will show how to work with the data locally, outside of the Earthdata Cloud (AWS us-west2). For more about working in the cloud, please reference _. First we'll show how to stream (directly access) data from each dataset, then we'll show how to authenticate and download. A download-based workflow typically fits better for larger files or older formats when running analyses outside of the cloud.

First, import the libraries we'll use to open the data. For EMIT we will use a module included in the repository to open and orthorectify the data.

In [ ]:
import xarray as xr
import rasterio as rio
import rioxarray as rxr
import hvplot.xarray
from modules.emit_tools import emit_xarray

To access data from NASA, you'll need to provide credentials. When streaming this can best be done using the token or cookies set up by the `earthaccess` library. Since we've already logged in, we can start an `fsspec` session to manage our connection to a remote file, including sending credentials. This allows other libraries to work with a URL as if its a local file.

In [ ]:
fs = earthaccess.get_fsspec_https_session()

From our results, we'll first open an AVIRIS-3 file. These files are in heirarchichal netcdf4 format, meaning they have multiple groups with datasets within each. To open all of the groups, we'll use the `open_datatree` function from xarray.

In [ ]:
aviris_file = fs.open(aviris_links[0])
aviris_ds = xr.open_datatree(aviris_file)
aviris_ds

This opens the file lazily, meaning we haven't loaded any data yet, just metadata. Before we load the data, we can perform some subsetting operations or select only the groups we want data from. For example, below we grab the `reflectance` variable from the `reflectance` group, select a wavelength, and plot using `hvplot`.

In [ ]:
aviris_850 = aviris_ds['reflectance']['reflectance'].sel(wavelength=850, method='nearest').compute()

In [ ]:
aviris_850.hvplot.image(x="easting", y="northing", cmap='viridis', frame_width=750, aspect='equal')

Now, lets open an ECOSTRESS L2T LSTE file. These are provided as cloud-optimized geotiffs. To open these, we can similarly use a session to manage our remote connection to a url. For `geotiff` files, the best performance for this is achieved by using a session from `rasterio`. Often its best to include a retry and retry delay. This will prevent your workflow from breaking if there are connectivity issues.

In [ ]:
# Cookies
env = rio.Env(
    GDAL_HTTP_COOKIEFILE="~/cookies.txt",
    GDAL_HTTP_COOKIEJAR="~/cookies.txt",
    GDAL_DISABLE_READDIR_ON_OPEN="EMPTY_DIR",
    GDAL_HTTP_MAX_RETRY="10",
    GDAL_HTTP_RETRY_DELAY="0.5",
)

Enter the rasterio session.

In [ ]:
# Enter our Rasterio Session for the rest of the notebook
env.__enter__()

Open the file, then squeeze to remove the 'band' dimension from the array since we only have one band.

In [ ]:
eco_ds = rxr.open_rasterio(eco_links[1], mask_and_scale=True).squeeze('band', drop=True)
eco_ds

Visualize using `hvplot.image`.

In [ ]:
eco_ds.hvplot.image(x='x',y='y',cmap='Spectral_r', aspect='equal', title="Surface Temperature (K)")

In [ ]:
# Exit rasterio session
env.__exit__

Similar to the AVIRIS-3 data, EMIT data is in .netcdf4 format, so we can use an `fsspec` session to stream the data. Unlike the AVIRIS-3 data, however, EMIT L2A Reflectance Version 1 data is not orthocorrected or chunked for streaming. This means its typically better to download this data unless you're working in the cloud. First we'll download a an EMIT scene from a our list, then we'll open, orthorectify the data, and visualize it. 

Import the `download_granules` function from `modules.search_util`. Then create a list containing a single url and use the function to download the scene.

In [ ]:
from modules.search_util import download_granules

single_emit_url_list = [emit_links[0]]
download_granules(url_list=single_emit_url_list,output_directory='../data/')

Now get the filepath to the local copy of the EMIT L2A RFL scene.

In [ ]:
emit_file = f'../data/{single_emit_url_list[0].split("/")[-1]}'

Open using the `emit_xarray` function, with the `ortho=True` argument to orthorectify the image.

In [ ]:
emit_ds = emit_xarray(emit_file, ortho=True)
emit_ds

In [ ]:
# Set fill values to np.nan
emit_ds['reflectance'].data[emit_ds['reflectance'].data==-9999] = np.nan

Select the band closest to 850nm and visualize using `hvplot` like we did for AVIRIS-3 Data.

In [ ]:
emit_850 = emit_ds['reflectance'].sel(wavelengths=850,method='nearest')

In [ ]:
#
emit_850.hvplot.image(x="longitude", y="latitude", cmap='viridis', frame_width=750, aspect='equal')

## Contact Info:  

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://lpdaac.usgs.gov/>  

¹Work performed under USGS contract 140G0121D0001 for NASA contract NNG14HH33I. 